In [39]:
import torch
from torch import nn
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision.transforms import ToTensor


In [40]:
training_dataset = datasets.FashionMNIST(
    root="./data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_dataset = datasets.FashionMNIST(
    root="./data",
    train=False,
    download=True,
    transform=ToTensor()
)


In [41]:
batch_size = 64
train_dataloader = DataLoader(training_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

for x, y in test_dataloader:
    print(x.shape, y.shape, y.dtype)
    break

torch.Size([64, 1, 28, 28]) torch.Size([64]) torch.int64


In [42]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using device: {device}")


class FirstNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = FirstNeuralNetwork().to(device)
print(model)

Using device: cuda
FirstNeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [43]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [44]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)

        prediction = model(x)
        loss = loss_fn(prediction, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(x)
            print(f"Loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [45]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            prediction = model(x)
            test_loss += loss_fn(prediction, y).item()
            correct += (prediction.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [46]:
epochs = 10
for t in range(epochs):
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Loss: 2.302527  [   64/60000]
Loss: 2.289273  [ 6464/60000]
Loss: 2.267577  [12864/60000]
Loss: 2.261112  [19264/60000]
Loss: 2.248986  [25664/60000]
Loss: 2.210430  [32064/60000]
Loss: 2.228735  [38464/60000]
Loss: 2.184390  [44864/60000]
Loss: 2.183722  [51264/60000]
Loss: 2.173040  [57664/60000]
Test Error: 
 Accuracy: 49.8%, Avg loss: 2.149966 

Loss: 2.154038  [   64/60000]
Loss: 2.149938  [ 6464/60000]
Loss: 2.090310  [12864/60000]
Loss: 2.109309  [19264/60000]
Loss: 2.065840  [25664/60000]
Loss: 1.999882  [32064/60000]
Loss: 2.035085  [38464/60000]
Loss: 1.950663  [44864/60000]
Loss: 1.951746  [51264/60000]
Loss: 1.911281  [57664/60000]
Test Error: 
 Accuracy: 57.1%, Avg loss: 1.888413 

Loss: 1.911435  [   64/60000]
Loss: 1.896695  [ 6464/60000]
Loss: 1.774809  [12864/60000]
Loss: 1.816883  [19264/60000]
Loss: 1.720483  [25664/60000]
Loss: 1.664071  [32064/60000]
Loss: 1.693677  [38464/60000]
Loss: 1.587568  [44864/60000]
Loss: 1.610034  [51264/60000]
Loss: 1.526778  [57664/600

64.8 - 5 epoch

70.8 - 10 epoch

In [47]:
torch.save(model.state_dict(), "model.pth")

use that state

In [48]:
model = FirstNeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

In [52]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()

x, y = test_dataset[100][0], test_dataset[100][1]
with torch.no_grad():
    x = x.to(device)
    prediction = model(x)
    predicted, actual = classes[prediction[0].argmax(0)], classes[y]
    print(f"Predicted: {predicted}, Actual: {actual}")

Predicted: Dress, Actual: Dress
